<a href="https://colab.research.google.com/github/teleyun/bigdata/blob/main/3%EC%9D%BC%EC%B0%A8_%EA%B3%BC%EC%A0%9C_%EC%9C%A4%ED%98%84%EC%A0%95_v4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 3일차 과제 

- 아래 문제는 3일차 수업에 사용한 데이터를 활용하여 시험 문제와 유사한 형태로 문제를 만든 것입니다.
- 수업 내용과 교재를 참고하여 실제 시험을 보는 것처럼 한 셀 안에 코드를 작성해보세요.
- 제출 방법: 6/1(수)까지 코드를 작성한 .ipynb 파일을 yi.kyeongsuk@gmail.com로 제출 (***단, 코드 실행 시 정답 csv 파일이 생성되어야 합니다.***)


In [2]:
# 이 셀을 먼저 실행한 후 문제를 풀어주세요.
!git clone https://github.com/kyeongsuk-yi/data_analyst/

Cloning into 'data_analyst'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 19 (delta 3), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (19/19), done.


### 문제
아래는 boston housing 데이터 세트를 변형시킨 데이터이다.   

1) 제공 데이터 목록    
*   housing_y_train.csv : 주택 고가 여부 데이터(학습용), csv 형식의 파일
*   housing_X_train.csv, housing_X_test.csv : 주택 정보(학습 및 평가용), csv 형식의 파일

2) 주어진 학습용 데이터 (housing_X_train.csv, housing_y_train.csv)를 활용하여 해당 주택이 고가인지 아닌지를 예측하는 모형을 만든 후, 이를 평가용 데이터(housing_X_test.csv)에 적용하여 얻는 주택 고가 여부(고가일 경우 1, 아니면 0)를 다음과 같은 형식의 csv 파일로 생성하시오. (제출한 모델의 성능은 ROC-AUC 평가 지표에 따라 채점).

|isHighValue|
|---|
|0|
|0|
|1|
|1|
|0|
|...|

3) 유의사항
- 성능이 우수한 예측모형을 구축하기 위해서는 적절한 데이터 전처리, feature engineering, 분류 알고리즘 사용, 초매개변수 최적화, 모형 앙상블 등이 수반되어야 한다. 
- 이름.csv (예: 홍길동.csv) 파일이 만들어지도록 코드를 제출한다.
- 제출한 모델의 성능은 ROC-AUC 평가 지표에 따라 채점한다. 

In [5]:
# 출력을 원하실 경우 print() 함수 활용
# 예시) print(df.head())

# getcwd(), chdir() 등 작업 폴더 설정 불필요
# 파일 경로 상 내부 드라이브 경로(C: 등) 접근 불가

# 데이터 파일 읽기 예제
import pandas as pd
X_test = pd.read_csv("./data_analyst/data/housing_X_test.csv")
X_train = pd.read_csv("./data_analyst/data/housing_X_train.csv")
y_train = pd.read_csv("./data_analyst/data/housing_y_train.csv")


######### 여기에 코드를 작성해 주세요. #########
# 결측치 처리
X_train['CHAS'] = X_train['CHAS'].fillna(X_train['CHAS'].mean())
X_train['LSTAT'] = X_train['LSTAT'].fillna(X_train['LSTAT'].mean())

# 이상치 처리
X_train_y_train = pd.concat([X_train, y_train], axis = 1)
Q1, Q3 = X_train_y_train['CHAS'].quantile([0.25, 0.75])
IQR = Q3 - Q1
upper_bound = Q3 + 1.5 * IQR
lower_bound = Q1 - 1.5 * IQR
X_train1 = X_train_y_train[((X_train_y_train['CHAS'] <= upper_bound) & (X_train_y_train['CHAS'] >= lower_bound))]
X_train2 = X_train1.loc[:, 'CRIM':'LSTAT']
y_train2 = X_train1['isHighValue']

# 학습, 검증용 데이터 분리
from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(X_train2, y_train2, test_size=0.3, random_state = 123)

# 최소최대정규화
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(X_tr)
X_tr_scaled = scaler.transform(X_tr)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# 분석모델 구축
from sklearn.ensemble import RandomForestClassifier

model_rfc = RandomForestClassifier(random_state = 123)
model_rfc.fit(X_tr_scaled, y_tr)

# 분석모형 평가
from sklearn.metrics import classification_report

y_pred = model_rfc.predict(X_val_scaled)
#print(classification_report(y_val, y_pred, labels=[0,1]))

from sklearn.metrics import roc_auc_score

y_pred_proba = model_rfc.predict_proba(X_val_scaled)[:,1]
#print(roc_auc_score(y_val, y_pred_proba))

# 분석모형 적용
pred = model_rfc.predict(X_test_scaled)

# 결과값 제출
pd.DataFrame({'isHighValue': pred}).to_csv('윤현정.csv', index=False)

################################################

# 답안 제출 참고
# 아래 코드 예측변수와 이름을 개인별로 변경하여 활용
# pd.DataFrame({'cust_id': X_test.cust_id, 'gender': pred}).to_csv('홍길동.csv', index=False)